# Threat Modeling Overview

Threat modeling is a structured approach to identifying, quantifying, and addressing security risks in systems. It helps organizations proactively discover vulnerabilities before attackers exploit them.

## Topics Covered
- **STRIDE & DREAD** - Threat classification and risk scoring frameworks
- **Data Flow Diagrams (DFDs)** - Visualizing system architecture for security analysis
- **Attack Trees** - Hierarchical threat decomposition
- **MITRE ATT&CK** - Industry-standard adversary tactics and techniques
- **Threat Intelligence** - Gathering and applying threat data
- **Risk Assessment Matrices** - Prioritizing threats by impact and likelihood

## 1. STRIDE Threat Model

**STRIDE** is a mnemonic developed by Microsoft for categorizing security threats. Each letter represents a threat category mapped to a security property violation.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           STRIDE THREAT MODEL                               │
├─────────────────┬───────────────────────┬───────────────────────────────────┤
│     Threat      │   Security Property   │            Description            │
│                 │       Violated        │                                   │
├─────────────────┼───────────────────────┼───────────────────────────────────┤
│  S - Spoofing   │   Authentication      │ Impersonating a user or system    │
├─────────────────┼───────────────────────┼───────────────────────────────────┤
│  T - Tampering  │   Integrity           │ Unauthorized data modification    │
├─────────────────┼───────────────────────┼───────────────────────────────────┤
│  R - Repudiation│   Non-repudiation     │ Denying performed actions         │
├─────────────────┼───────────────────────┼───────────────────────────────────┤
│  I - Info       │   Confidentiality     │ Unauthorized data exposure        │
│    Disclosure   │                       │                                   │
├─────────────────┼───────────────────────┼───────────────────────────────────┤
│  D - Denial of  │   Availability        │ Making system unavailable         │
│      Service    │                       │                                   │
├─────────────────┼───────────────────────┼───────────────────────────────────┤
│  E - Elevation  │   Authorization       │ Gaining unauthorized privileges   │
│    of Privilege │                       │                                   │
└─────────────────┴───────────────────────┴───────────────────────────────────┘
```

### STRIDE Application Process
1. **Decompose** the application into components
2. **Identify** data flows and trust boundaries
3. **Apply** each STRIDE category to each component
4. **Document** identified threats and mitigations

## 2. DREAD Risk Assessment

**DREAD** is a risk scoring model that quantifies the severity of identified threats. Each factor is rated 1-10.

| Factor | Description | Scoring Criteria |
|--------|-------------|------------------|
| **D**amage | Potential impact if exploited | 1=Minimal, 10=Complete system compromise |
| **R**eproducibility | Ease of reproducing the attack | 1=Very hard, 10=Always reproducible |
| **E**xploitability | Skill/tools needed to exploit | 1=Expert+custom tools, 10=Script kiddie |
| **A**ffected Users | Scope of impact | 1=Single user, 10=All users |
| **D**iscoverability | Ease of finding the vulnerability | 1=Nearly impossible, 10=Publicly known |

### DREAD Score Calculation
```
Risk Score = (D + R + E + A + D) / 5

Risk Levels:
  • 1-3:   Low Risk     → Monitor, address in future releases
  • 4-6:   Medium Risk  → Plan remediation within sprint
  • 7-10:  High Risk    → Immediate action required
```

### Example: SQL Injection Vulnerability
| Factor | Score | Justification |
|--------|-------|---------------|
| Damage | 9 | Full database access, data exfiltration |
| Reproducibility | 10 | Consistent exploitation |
| Exploitability | 8 | Automated tools available (sqlmap) |
| Affected Users | 10 | All application users |
| Discoverability | 7 | Common vulnerability scanners detect it |
| **Average** | **8.8** | **HIGH RISK** |

## 3. Data Flow Diagrams (DFD)

DFDs visualize how data moves through a system, highlighting **trust boundaries** where security controls are needed.

### DFD Elements
```
┌──────────────────────────────────────────────────────────────────────────┐
│  Symbol      │  Name              │  Description                         │
├──────────────┼────────────────────┼──────────────────────────────────────┤
│  [  User  ]  │  External Entity   │  Outside actors (users, systems)    │
│  (Process)   │  Process           │  Code that transforms data          │
│  ==Data==    │  Data Store        │  Databases, files, caches           │
│  ──────►     │  Data Flow         │  Movement of data between elements  │
│  - - - - -   │  Trust Boundary    │  Privilege/security level change    │
└──────────────┴────────────────────┴──────────────────────────────────────┘
```

### Example: Web Application DFD
```
                         TRUST BOUNDARY
                    - - - - - - - - - - - - - - - - - - - - - - - -
                    :                                             :
  [  Browser  ]     :    ( Web Server )     ( App Logic )         :
       │            :          │                  │               :
       │ HTTPS      :          │                  │               :
       ├────────────:──────────►                  │               :
       │            :          │    API Call      │               :
       │            :          ├──────────────────►               :
       │            :          │                  │               :
       │            :          │                  │  SQL Query    :
       │            :          │                  ├───────────────:───►  ==Database==
       │            :          │                  │               :
       ◄────────────:──────────┤◄─────────────────┤               :
       │  Response  :          │    Response      │               :
                    :                                             :
                    - - - - - - - - - - - - - - - - - - - - - - - -
                              DMZ / Internal Network
```

**Key Security Questions at Each Boundary:**
- Is data encrypted in transit?
- Is input validated before crossing boundaries?
- Are credentials properly managed?

## 4. Attack Trees

Attack trees model the steps an attacker might take to achieve a goal. The root is the attacker's objective; branches are attack paths.

### Attack Tree: Steal User Credentials
```
                        ┌─────────────────────────┐
                        │   STEAL USER CREDS      │  (Root Goal)
                        │   [AND/OR Node]         │
                        └───────────┬─────────────┘
                                    │
            ┌───────────────────────┼───────────────────────┐
            │ OR                    │ OR                    │ OR
            ▼                       ▼                       ▼
  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
  │  Phishing       │    │  Exploit Vuln   │    │  Social Eng     │
  │  Attack         │    │  in Auth System │    │  Helpdesk       │
  └────────┬────────┘    └────────┬────────┘    └─────────────────┘
           │                      │
     ┌─────┴─────┐          ┌─────┴─────┐
     │ AND       │ AND      │ OR        │ OR
     ▼           ▼          ▼           ▼
  ┌──────┐  ┌────────┐  ┌───────┐  ┌──────────┐
  │Create│  │Victim  │  │SQL    │  │Brute     │
  │Fake  │  │Clicks  │  │Inject │  │Force     │
  │Login │  │Link    │  │Login  │  │Password  │
  └──────┘  └────────┘  └───────┘  └──────────┘
```

### Attack Tree Annotations
| Node | Cost | Difficulty | Detection Risk |
|------|------|------------|----------------|
| Phishing | Low | Medium | Medium |
| SQL Injection | Low | Low (if vuln exists) | High |
| Brute Force | Low | High (rate limiting) | Very High |
| Social Engineering | Low | Medium | Low |

## 5. MITRE ATT&CK Framework

**ATT&CK** (Adversarial Tactics, Techniques, and Common Knowledge) is a globally-accessible knowledge base of adversary behaviors based on real-world observations.

### ATT&CK Matrix Structure

| Tactic (WHY) | Example Techniques (HOW) |
|--------------|-------------------------|
| **Reconnaissance** | Active Scanning, Phishing for Information |
| **Resource Development** | Acquire Infrastructure, Develop Capabilities |
| **Initial Access** | Phishing, Exploit Public-Facing App, Supply Chain |
| **Execution** | Command-Line Interface, PowerShell, Scripting |
| **Persistence** | Registry Run Keys, Scheduled Tasks, Web Shell |
| **Privilege Escalation** | Exploitation, Access Token Manipulation |
| **Defense Evasion** | Obfuscation, Disable Security Tools, Rootkits |
| **Credential Access** | Credential Dumping, Brute Force, Keylogging |
| **Discovery** | Network Scanning, Account Discovery |
| **Lateral Movement** | Remote Services, Pass-the-Hash, SSH Hijacking |
| **Collection** | Screen Capture, Data from Local System |
| **Exfiltration** | Exfil Over C2 Channel, Cloud Storage |
| **Impact** | Data Destruction, Ransomware, Defacement |

### Using ATT&CK for Defense
1. **Map** existing security controls to ATT&CK techniques
2. **Identify** gaps in detection/prevention coverage
3. **Prioritize** based on threat intelligence (which TTPs target your industry)
4. **Test** controls using ATT&CK-based red team exercises

## 6. Threat Intelligence Basics

Threat intelligence is evidence-based knowledge about existing or emerging threats that informs security decisions.

### Intelligence Types

| Level | Audience | Content | Refresh Rate |
|-------|----------|---------|-------------|
| **Strategic** | Executives, Board | Threat landscape trends, business risk | Quarterly |
| **Tactical** | Security Architects | TTPs, threat actor profiles | Monthly |
| **Operational** | SOC Analysts, IR | Campaign details, attack timing | Weekly |
| **Technical** | Detection Engineers | IOCs (IPs, hashes, domains) | Real-time |

### Key Threat Intelligence Sources
- **Open Source (OSINT)**: VirusTotal, Shodan, AlienVault OTX
- **Commercial Feeds**: Recorded Future, CrowdStrike, Mandiant
- **ISACs**: Industry-specific sharing (FS-ISAC, H-ISAC)
- **Government**: CISA Alerts, FBI Flash Reports, NCSC

### Threat Intelligence Lifecycle
```
Direction → Collection → Processing → Analysis → Dissemination → Feedback
    ▲                                                              │
    └──────────────────────────────────────────────────────────────┘
```

## 7. Risk Assessment Matrix

A risk matrix combines **likelihood** and **impact** to prioritize threats for remediation.

### Standard 5x5 Risk Matrix
```
                              IMPACT
              │ Negligible │   Minor   │  Moderate │   Major   │ Catastrophic │
              │     (1)    │    (2)    │    (3)    │    (4)    │     (5)      │
 ─────────────┼────────────┼───────────┼───────────┼───────────┼──────────────┤
 Almost       │            │           │           │           │              │
 Certain (5)  │   MEDIUM   │   HIGH    │   HIGH    │  CRITICAL │   CRITICAL   │
 ─────────────┼────────────┼───────────┼───────────┼───────────┼──────────────┤
 Likely (4)   │    LOW     │  MEDIUM   │   HIGH    │   HIGH    │   CRITICAL   │
 ─────────────┼────────────┼───────────┼───────────┼───────────┼──────────────┤
L Possible(3) │    LOW     │  MEDIUM   │  MEDIUM   │   HIGH    │    HIGH      │
I ────────────┼────────────┼───────────┼───────────┼───────────┼──────────────┤
K Unlikely(2) │    LOW     │    LOW    │  MEDIUM   │  MEDIUM   │    HIGH      │
E ────────────┼────────────┼───────────┼───────────┼───────────┼──────────────┤
L Rare (1)    │    LOW     │    LOW    │    LOW    │  MEDIUM   │   MEDIUM     │
I ────────────┴────────────┴───────────┴───────────┴───────────┴──────────────┘
H
O   Risk Score = Likelihood × Impact
O   
D   Response Guidelines:
      CRITICAL (20-25): Immediate executive escalation, stop-work authority
      HIGH (12-19):     Remediate within 7 days, assign dedicated owner
      MEDIUM (6-11):    Remediate within 30 days, include in sprint
      LOW (1-5):        Accept, monitor, or address opportunistically
```

### Risk Treatment Options
| Strategy | Description | Example |
|----------|-------------|--------|
| **Mitigate** | Reduce likelihood or impact | Implement WAF, patch systems |
| **Transfer** | Shift risk to third party | Cyber insurance, outsource |
| **Accept** | Acknowledge and monitor | Low-risk items with high remediation cost |
| **Avoid** | Eliminate the risk source | Discontinue vulnerable service |

## Key Takeaways

| Framework | Best Used For | Key Output |
|-----------|--------------|------------|
| **STRIDE** | Systematic threat identification | Categorized threat list |
| **DREAD** | Risk prioritization | Numerical risk scores |
| **DFD** | Visualizing attack surfaces | Trust boundary diagrams |
| **Attack Trees** | Understanding attack paths | Hierarchical attack models |
| **MITRE ATT&CK** | Mapping adversary behavior | Detection coverage gaps |
| **Threat Intel** | Proactive defense | Actionable IOCs and TTPs |
| **Risk Matrix** | Prioritizing remediation | Treatment decisions |

### Threat Modeling Process Flow
```
┌──────────────┐   ┌──────────────┐   ┌──────────────┐   ┌──────────────┐
│  1. Scope    │──►│  2. Model    │──►│  3. Identify │──►│  4. Assess   │
│  & Decompose │   │  with DFD    │   │  via STRIDE  │   │  with DREAD  │
└──────────────┘   └──────────────┘   └──────────────┘   └──────────────┘
                                                                 │
┌──────────────┐   ┌──────────────┐   ┌──────────────┐           │
│  7. Iterate  │◄──│  6. Validate │◄──│  5. Mitigate │◄──────────┘
│  & Update    │   │  & Test      │   │  & Document  │
└──────────────┘   └──────────────┘   └──────────────┘
```

**Resources:**
- [MITRE ATT&CK](https://attack.mitre.org/)
- [Microsoft Threat Modeling Tool](https://aka.ms/threatmodelingtool)
- [OWASP Threat Dragon](https://owasp.org/www-project-threat-dragon/)

# Threat Modeling - Overview

Threat modeling is a structured way to identify, prioritize, and mitigate security risks before they become incidents. It turns "what could go wrong?" into concrete threats, assumptions, and controls.

## Purpose
- Reduce security surprises by surfacing risks early.
- Align engineering, security, and product on what matters most.
- Produce actionable mitigations tied to system components.

## Key questions this section answers
- What are the critical assets and trust boundaries?
- Which threats are most likely and most damaging?
- What controls reduce risk to an acceptable level?

## Core frameworks
- STRIDE (spoofing, tampering, repudiation, information disclosure, denial of service, elevation of privilege)
- LINDDUN for privacy threats
- PASTA and Trike for risk-driven analysis
- Attack trees and MITRE ATT&CK for adversary behaviors

## Outputs you should expect
- A data flow diagram (DFD) with trust boundaries
- A threat register with likelihood, impact, and mitigations
- Clear ownership and follow-up actions


## Process at a glance
1. Define scope and assets.
2. Map data flows and trust boundaries.
3. Enumerate threats by component and data flow.
4. Rate risk and prioritize.
5. Choose mitigations and validate with tests.
6. Review changes as the system evolves.


## Risk scoring quick model
A simple expected-loss proxy:

$$Risk = Likelihood \times Impact$$

Use a 1-5 scale for each and focus on high-risk items first. The goal is consistency, not false precision.


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

threats = pd.DataFrame(
    [
        {
            "component": "Web app",
            "threat": "Credential stuffing",
            "category": "Spoofing",
            "likelihood": 4,
            "impact": 4,
            "mitigation": "MFA, rate limits, bot detection",
        },
        {
            "component": "API",
            "threat": "Parameter tampering",
            "category": "Tampering",
            "likelihood": 3,
            "impact": 4,
            "mitigation": "Schema validation, authZ checks",
        },
        {
            "component": "Database",
            "threat": "Privilege escalation",
            "category": "Elevation of privilege",
            "likelihood": 2,
            "impact": 5,
            "mitigation": "Least privilege, row-level policies",
        },
        {
            "component": "Auth service",
            "threat": "Token replay",
            "category": "Repudiation",
            "likelihood": 3,
            "impact": 3,
            "mitigation": "Short TTL, nonce, audit logs",
        },
        {
            "component": "Edge",
            "threat": "Volumetric DDoS",
            "category": "Denial of service",
            "likelihood": 3,
            "impact": 5,
            "mitigation": "WAF, autoscale, rate limits",
        },
    ]
)

threats["risk"] = threats["likelihood"] * threats["impact"]


def bucket(score: int) -> str:
    if score >= 16:
        return "critical"
    if score >= 12:
        return "high"
    if score >= 8:
        return "medium"
    return "low"


threats["level"] = threats["risk"].map(bucket)
threats.sort_values("risk", ascending=False)


In [ ]:
import numpy as np

likelihood = np.arange(1, 6)
impact = np.arange(1, 6)
score = np.outer(likelihood, impact)

fig = go.Figure(
    data=go.Heatmap(
        z=score,
        x=impact,
        y=likelihood,
        colorscale="YlOrRd",
        colorbar=dict(title="Risk"),
    )
)
fig.update_layout(
    title="Risk matrix (likelihood x impact)",
    xaxis_title="Impact",
    yaxis_title="Likelihood",
    yaxis_autorange="reversed",
)
fig

## STRIDE mapping example
Map each component to the STRIDE categories to force breadth of thinking. The goal is to list concrete threats and then map them to mitigations.


In [ ]:
stride = pd.DataFrame(
    [
        {
            "component": "Web app",
            "stride": "Spoofing",
            "example": "Session hijacking",
            "mitigation": "Secure cookies, CSRF protection",
        },
        {
            "component": "Web app",
            "stride": "Information disclosure",
            "example": "Leaky error pages",
            "mitigation": "Generic error responses, logging policy",
        },
        {
            "component": "API",
            "stride": "Tampering",
            "example": "Insecure direct object access",
            "mitigation": "AuthZ checks per resource",
        },
        {
            "component": "API",
            "stride": "Repudiation",
            "example": "No audit trail",
            "mitigation": "Immutable audit logs",
        },
        {
            "component": "Database",
            "stride": "Elevation of privilege",
            "example": "Wide service account rights",
            "mitigation": "Least privilege roles",
        },
    ]
)

stride


## Data flow diagram (simplified)
Use a data flow diagram to identify trust boundaries, external dependencies, and risky data paths. The shapes below are a lightweight example you can adapt to your system.


In [ ]:
nodes = {
    "User": (0, 0),
    "Web app": (1, 0),
    "API": (2, 0),
    "Auth": (2, 1),
    "Database": (3, 0),
}

edges = [
    ("User", "Web app"),
    ("Web app", "API"),
    ("API", "Database"),
    ("API", "Auth"),
]

fig = go.Figure()

for start, end in edges:
    x0, y0 = nodes[start]
    x1, y1 = nodes[end]
    fig.add_trace(
        go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            mode="lines",
            line=dict(color="#6b7280", width=2),
            hoverinfo="none",
            showlegend=False,
        )
    )

fig.add_trace(
    go.Scatter(
        x=[pos[0] for pos in nodes.values()],
        y=[pos[1] for pos in nodes.values()],
        text=list(nodes.keys()),
        mode="markers+text",
        textposition="bottom center",
        marker=dict(size=18, color="#2563eb"),
        showlegend=False,
    )
)

fig.add_shape(
    type="rect",
    x0=0.5,
    y0=-0.5,
    x1=3.5,
    y1=1.5,
    line=dict(color="#ef4444", dash="dash"),
)

fig.update_layout(
    title="Example data flow with a trust boundary",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    margin=dict(l=20, r=20, t=50, b=20),
)
fig

## Mitigations and controls
- Preventive: authN/authZ, least privilege, input validation, secure defaults.
- Detective: logging, alerting, anomaly detection, audit trails.
- Resilient: rate limits, circuit breakers, autoscaling, graceful degradation.
- Recovery: backups, incident runbooks, tabletop exercises.


## Practical tips
- Keep models small and current; update on architecture changes.
- Pair security and engineering in the same session to avoid blind spots.
- Tie every mitigation to an owner and a concrete test.

## Exercises
1. Draw a DFD for a login flow and identify two trust boundaries.
2. Produce a STRIDE table for a file upload service.
3. Convert your top three risks into specific backlog items.

## Further reading
- STRIDE, LINDDUN, PASTA, Trike, and MITRE ATT&CK references
- NIST risk management guidance (SP 800-30)
- OWASP ASVS for control-level checklists
